# TinyDB

TinyDB is a small and lightweight [NoSQL](http://en.wikipedia.org/wiki/NoSQL) database framework based on simple JSON files.

## Source

Official Website:
- [getting started](http://tinydb.readthedocs.org/en/latest/getting-started.html)
- [advanced usage](http://tinydb.readthedocs.org/en/latest/usage.html)

## Code

Some examples to create a database and insert, delete and seach for elements.

### Basics

Generate new database. If `path` leads to an existing file, the data is read. Otherwise a new database is created.

In [1]:
path = './testData.json'
from tinydb import TinyDB, where
db = TinyDB(path)

Insert some data into `db`.

In [2]:
print(db.insert({'a':1, 'b':3})) # return the elements id [1,2,..]
db.all() # returns list of dicts

159


[{'petal length (cm)': 1.4,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 5.1,
  'sepal width (cm)': 3.5},
 {'petal length (cm)': 1.4,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 4.9,
  'sepal width (cm)': 3.0},
 {'petal length (cm)': 1.3,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 4.7,
  'sepal width (cm)': 3.2},
 {'petal length (cm)': 1.5,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 4.6,
  'sepal width (cm)': 3.1},
 {'petal length (cm)': 1.4,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 5.0,
  'sepal width (cm)': 3.6},
 {'petal length (cm)': 1.7000000000000002,
  'petal width (cm)': 0.4,
  'sepal length (cm)': 5.4,
  'sepal width (cm)': 3.9},
 {'petal length (cm)': 1.4,
  'petal width (cm)': 0.30000000000000004,
  'sepal length (cm)': 4.6,
  'sepal width (cm)': 3.4},
 {'petal length (cm)': 1.5,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 5.0,
  'sepal width (cm)': 3.4},
 {'petal length (cm)': 1.4,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 4.4

Fill with some data ([iris](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)).

In [3]:
db.purge()

from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
keys = iris.feature_names
print('== data ==\n', data[:5], '\n ...')
print('== keys ==\n', keys)

== data ==
 [[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]] 
 ...
== keys ==
 ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [4]:
if(0 == len(db)):
    for d in data:
        db.insert({
            keys[0]: d[0],
            keys[1]: d[1],
            keys[2]: d[2],
            keys[3]: d[3]
        })
# or
# db.insert_multiple([ {}, {}, ...]) # returns list of ids
db.search(where(keys[0]) >= 7.6 and where(keys[1]) >= 4.2)

[{'petal length (cm)': 1.5,
  'petal width (cm)': 0.4,
  'sepal length (cm)': 5.7,
  'sepal width (cm)': 4.4},
 {'petal length (cm)': 1.4,
  'petal width (cm)': 0.2,
  'sepal length (cm)': 5.5,
  'sepal width (cm)': 4.2}]

### Error Handling

In [5]:
try:
    r = db.search(where(keys[0]) > 1000)[0]
    print(r)
except:
    print('Not in db.')

Not in db.


In [6]:
r = db.get(where(keys[0]) > 1000)
if not r:
    print('Not in db.') # r is None

Not in db.


If the actual data is irrelevant, to check for the existance of an element, use `contains` or `count`.

In [7]:
db.contains(where(keys[2]) == 1)

True

In [8]:
db.count(where(keys[2]) == 1)

1

### IDs

In [9]:
elem = db.get(where(keys[2]) == 1)
elem.eid

23

In [10]:
if db.contains(eids=[11, 12]):
    e1 = db.get(eid=11)
    e2 = db.get(eid=12)
    db.remove(eids=[11, 12])
    db.insert_multiple([e1, e2])

print(len(db))    
# db.remove(eids=list(arange(70, 75)))
# print(len(db))

150


### Regex and nested queries

In [11]:
# db.remove(where('field').has('name').has('last_name') == 'Doe')
db.insert({'field': {'name': {'first_name': 'John', 'last_name': 'Doe'}}})
# print(db.search(where('field.name.last_name') == 'Doe'))
# print(db.search(where('field.name.last_name').matches('[0-9]*')),'\n')

db.remove(where('field').any(where('val') == 1))
db.insert({'field': [{'val': 1}, {'val': 2}, {'val': 3}]})
print(db.search(where('field').any(where('val') == 1)))

[{'field': [{'val': 1}, {'val': 2}, {'val': 3}]}]
